In [20]:
# Set columns in display
# pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns') # Reset options to default

# Set rows in display
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows') # Reset options to default

# Load Util File with TARE Model Functions

In [21]:
import os

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

# Get the current working directory of the project
project_root = os.path.abspath(os.getcwd())
print(f"Project root directory: {project_root}")

# Relative path to the file from the project root
relative_path = r"tare_model_functions_v1.4.1.ipynb"

# Construct the absolute path to the file
file_path = os.path.join(project_root, relative_path)
print(f"File path: {file_path}")

# Run the notebook and import variables
if os.path.exists(relative_path):
    get_ipython().run_line_magic('run', f'-i "{relative_path}"')
    print("Loaded All TARE Model Functions")
else:
    print(f"File not found: {relative_path}")

Project root directory: c:\Users\14128\Research\cmu-tare-model
File path: c:\Users\14128\Research\cmu-tare-model\tare_model_functions_v1.4.1.ipynb
Loaded All TARE Model Functions


In [22]:
# Storing Result Outputs in output_results folder
relative_path = r"output_results"
output_folder_path = os.path.join(project_root, relative_path)
print(f"Result outputs will be exported here: {output_folder_path}")

Result outputs will be exported here: c:\Users\14128\Research\cmu-tare-model\output_results


# Simulate Residential Energy Consumption using NREL End-Use Savings Shapes
- Filter EUSS Data: Only occupied units and Single Family Homes



In [23]:
# The ``inline`` flag will use the appropriate backend to make figures appear inline in the notebook.  
%matplotlib inline

import pandas as pd
import numpy as np

# `plt` is an alias for the `matplotlib.pyplot` module
import matplotlib.pyplot as plt

# import seaborn library (wrapper of matplotlib)
import seaborn as sns
sns.set(style="darkgrid")

# For regex, import re
import re

from datetime import datetime

# Get the current datetime
# Start the timer
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [24]:
print("""
-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------
BASELINE (Measure Package 0)
-------------------------------------------------------------------------------------------------------
""")

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

filename = "baseline_metadata_and_annual_results.csv"
relative_path = os.path.join(r"euss_data\resstock_amy2018_release_1.1\state", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

print("""
-------------------------------------------------------------------------------------------------------
Data Filters: Only occupied units and Single Family Homes
-------------------------------------------------------------------------------------------------------
""")

# Fix DtypeWarning error in columns:
# 'in.neighbors', 'in.geometry_stories_low_rise', 'in.iso_rto_region', 'in.pv_orientation', 'in.pv_system_size'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_baseline = pd.read_csv(file_path, dtype=columns_to_string)
occupancy_filter = df_euss_am_baseline['in.vacancy_status'] == 'Occupied'
df_euss_am_baseline = df_euss_am_baseline.loc[occupancy_filter]

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_baseline['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_baseline = df_euss_am_baseline.loc[house_type_filter]
# df_euss_am_baseline


-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------
BASELINE (Measure Package 0)
-------------------------------------------------------------------------------------------------------

Retrieved data for filename: baseline_metadata_and_annual_results.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\euss_data\resstock_amy2018_release_1.1\state\baseline_metadata_and_annual_results.csv

------------------------

In [25]:
# # Create a location ID for the name of the batch conversion file
# while True:
#     if menu_state == 'N':
#         location_id = 'National'
#         print("You chose to analyze all of the United States.")
#         break
#     elif menu_state == 'Y':
#         if menu_city == 'N':
#             try:
#                 location_id = str(input_state)
#                 print(f"Location ID is: {location_id}")
#                 break
#             except ValueError:
#                 print("Invalid input for state!")
#         elif menu_city == 'Y':
#             try:
#                 location_id = input_cityFilter.replace(', ', '_').strip()
#                 print(f"Location ID is: {location_id}")
#                 break
#             except AttributeError:
#                 print("Invalid input for city filter!")
#         else:
#             print("Incorrect state or city filter assignment!")
#     else:
#         print("Invalid data location. Check your inputs at the beginning of this notebook!")

In [26]:
# # Make a copy of the dataframe
# df_euss_am_baseline = df_euss_am_baseline.copy()

# Choose between national or sub-national level analysis
menu_state = get_menu_choice(menu_prompt, {'N', 'Y'})   # This code is only run in baseline

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'
    location_id = 'National'

# Filter down to state or city
else:
    input_state = get_state_choice(df_euss_am_baseline)    
    print(f"You chose to filter for: {input_state}")

    location_id = str(input_state)
    print(f"Location ID is: {location_id}")


    state_filter = df_euss_am_baseline['in.state'].eq(input_state)
    df_euss_am_baseline = df_euss_am_baseline.loc[state_filter]

    print(city_prompt)
    print(df_euss_am_baseline['in.city'].value_counts())

    menu_city = get_menu_choice(city_menu_prompt, {'N', 'Y'})

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
        location_id = str(input_state)
        print(f"Location ID is: {location_id}")
        
    # Filter to a city within the selected state
    else:
        input_cityFilter = get_city_choice(df_euss_am_baseline, input_state)
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")

        location_id = input_cityFilter.replace(', ', '_').strip()
        print(f"Location ID is: {location_id}")

        city_filter = df_euss_am_baseline['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_baseline = df_euss_am_baseline.loc[city_filter]

# Display the filtered dataframe
df_euss_am_baseline

You chose to analyze all of the United States.


,bldg_id,upgrade,weight,applicability,in.sqft,in.ahs_region,in.ashrae_iecc_climate_zone_2004,in.ashrae_iecc_climate_zone_2004_2_a_split,in.bathroom_spot_vent_hour,in.bedrooms,...,out.emissions.natural_gas.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.propane.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.electricity.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.fuel_oil.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.natural_gas.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.propane.lrmer_mid_case_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_15_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_low_re_cost_25_2025_start.co2e_kg,out.emissions.all_fuels.lrmer_mid_case_15_2025_start.co2e_kg
2,239,0,242.131013,True,1690.0,Non-CBSA East South Central,3A,3A,Hour20,3,...,215.943534,0.000000,8773.384074,0.0,215.943534,0.000000,3565.038262,6416.193347,5755.373221,8989.327608
3,273,0,242.131013,True,1690.0,Non-CBSA East South Central,3A,3A,Hour12,3,...,0.000000,0.000000,11296.731129,0.0,0.000000,0.000000,4339.903757,8193.678510,7254.665194,11296.731129
4,307,0,242.131013,True,1220.0,Non-CBSA East South Central,3A,3A,Hour0,4,...,0.000000,0.000000,8750.011820,0.0,0.000000,0.000000,3345.187937,6249.625611,5587.946834,8750.011820
5,409,0,242.131013,True,1220.0,Non-CBSA East South Central,3A,3A,Hour20,2,...,1642.477930,0.000000,5725.103641,0.0,1642.477930,0.000000,3784.993820,5489.549041,5164.458936,7367.581571
7,517,0,242.131013,True,1220.0,Non-CBSA East South Central,3A,3A,Hour1,3,...,0.000000,0.000000,8932.439414,0.0,0.000000,0.000000,3441.414837,6415.303853,5721.073473,8932.439414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548907,548226,0,242.131013,True,2176.0,Non-CBSA Mountain,6B,6B,Hour3,4,...,15324.356044,0.000000,2313.298407,0.0,15324.356044,0.000000,16802.023908,16545.095582,16472.920871,17637.654451
548908,548228,0,242.131013,True,1690.0,Non-CBSA Mountain,6B,6B,Hour6,4,...,8192.601682,0.000000,1889.439924,0.0,8192.601682,0.000000,9394.122057,9195.903552,9129.578822,10082.041606
548910,548417,0,242.131013,True,885.0,Non-CBSA Mountain,6B,6B,Hour18,2,...,5212.758359,0.000000,2112.907195,0.0,5212.758359,0.000000,6546.826589,6318.763521,6253.986448,7325.665554
548914,549740,0,242.131013,True,1220.0,Non-CBSA Mountain,7B,7B,Hour4,2,...,0.000000,268.627834,11423.104685,0.0,0.000000,268.627834,7173.561517,5931.433285,4609.821155,11691.732519


## Project Future Energy Consumption Using EIA Heating Degree Day (HDD) Forecasted Data (Factors)

In [27]:
# Factors for 2022 to 2050
filename = 'aeo_projections_2022_2050.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_hdd_projection_factors = pd.read_excel(io=file_path, sheet_name='hdd_factors_2022_2050')

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Convert the factors dataframe into a lookup dictionary
hdd_factor_lookup = df_hdd_projection_factors.set_index(['census_division']).to_dict('index')
hdd_factor_lookup

Retrieved data for filename: aeo_projections_2022_2050.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\aeo_projections_2022_2050.xlsx


{'National': {2022: 1,
  2023: 1.0028349414260749,
  2024: 0.9389536266963965,
  2025: 0.9344844368179533,
  2026: 0.9300828169743566,
  2027: 0.9257070259326153,
  2028: 0.9212779053519207,
  2029: 0.9168538581973203,
  2030: 0.9124351151864318,
  2031: 0.9080377039911245,
  2032: 0.9036541297129915,
  2033: 0.8992492812396443,
  2034: 0.894875507855348,
  2035: 0.8904758482849783,
  2036: 0.8860390020882589,
  2037: 0.8817285900905196,
  2038: 0.877365886428882,
  2039: 0.8729314040841085,
  2040: 0.8685839209369028,
  2041: 0.8642702226890459,
  2042: 0.8599120736340495,
  2043: 0.8555441810694344,
  2044: 0.8511753084862802,
  2045: 0.8468232704962843,
  2046: 0.8425090534289743,
  2047: 0.8382247585710751,
  2048: 0.8339389072548168,
  2049: 0.8297055204635582,
  2050: 0.8255002687057338},
 'East North Central': {2022: 1,
  2023: 0.9811731756651626,
  2024: 0.9307608526528707,
  2025: 0.928426948809709,
  2026: 0.9262486385560915,
  2027: 0.9239147347129298,
  2028: 0.921580830869

In [28]:
print("""
-------------------------------------------------------------------------------------------------------
Baseline Consumption:
-------------------------------------------------------------------------------------------------------
""")

# df_baseline_enduse(df_baseline, df_enduse, category, fuel_filter='Yes', tech_filter='Yes')
df_euss_am_baseline_home = df_enduse_refactored(df_baseline = df_euss_am_baseline,
                                                fuel_filter = 'Yes',
                                                tech_filter = 'Yes')

# Project Future Energy Consumption
df_euss_am_baseline_home = project_future_consumption(df=df_euss_am_baseline_home, hdd_factor_lookup=hdd_factor_lookup, menu_mp=menu_mp)
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Baseline Consumption:
-------------------------------------------------------------------------------------------------------

Processing column: in.clothes_dryer
Initial data types: object
Data types after processing: object
Processing column: in.cooking_range
Initial data types: object
Data types after processing: object
331531 rows remain after applying total heating consumption calculation
Filtered for the following fuels: ['Natural Gas', 'Electricity', 'Propane', 'Fuel Oil']
321357 rows remain after applying heating fuel filter
Filtered for the following Heating technologies: ['Electricity ASHP', 'Electricity Baseboard', 'Electricity Electric Boiler', 'Electricity Electric Furnace', 'Fuel Oil Fuel Boiler', 'Fuel Oil Fuel Furnace', 'Natural Gas Fuel Boiler', 'Natural Gas Fuel Furnace', 'Propane Fuel Boiler', 'Propane Fuel Furnace']
291558 rows remain after applying heating techn

,bldg_id,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,state,city,county,...,baseline_2029_cooking_consumption,baseline_2030_cooking_consumption,baseline_2031_cooking_consumption,baseline_2032_cooking_consumption,baseline_2033_cooking_consumption,baseline_2034_cooking_consumption,baseline_2035_cooking_consumption,baseline_2036_cooking_consumption,baseline_2037_cooking_consumption,baseline_2038_cooking_consumption
2,239,1690.0,South,East South Central,East South Central,Hot-Humid,90,AL,Not in a census Place,G0100390,...,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44,979.44
3,273,1690.0,South,East South Central,East South Central,Mixed-Humid,90,AL,In another census Place,G0100150,...,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20
4,307,1220.0,South,East South Central,East South Central,Hot-Humid,90,AL,Not in a census Place,G0100850,...,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20,537.20
5,409,1220.0,South,East South Central,East South Central,Hot-Humid,90,AL,Not in a census Place,G0100050,...,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63,1009.63
7,517,1220.0,South,East South Central,East South Central,Mixed-Humid,89,AL,In another census Place,G0101270,...,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18,377.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548905,548109,1690.0,West,Mountain,Mountain North,Cold,23,WY,In another census Place,G5600050,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61
548907,548226,2176.0,West,Mountain,Mountain North,Cold,23,WY,In another census Place,G5600050,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61
548908,548228,1690.0,West,Mountain,Mountain North,Cold,24,WY,Not in a census Place,G5600010,...,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61,405.61
548910,548417,885.0,West,Mountain,Mountain North,Cold,24,WY,Casper,G5600250,...,486.50,486.50,486.50,486.50,486.50,486.50,486.50,486.50,486.50,486.50


# Public Perspective: Monetized Marginal Damages from Emissions

## Step 1: Calculate emissions factors for different fuel sources

In [29]:
print("""
-------------------------------------------------------------------------------------------------------
Public Perspective: Monetized Marginal Damages from Emissions
-------------------------------------------------------------------------------------------------------
Step 1: Calculate emissions factors for different fuel sources
- Electricity
- Natural Gas
- Fuel Oil 
- Propane
      
Step 2: Adjust Natural Gas & Electricity Emissions Factors for Natural Gas Leakage
- CAMBIUM includes fugitive emissions along with other pre-combustion and combustion emissions
- Natural Gas Leakage: Deetjen et al 2021
      
Step 3: Multiply Emissions by Marginal Social Cost
- Only CO2e in the updated methods due to data quality issues for 
-------------------------------------------------------------------------------------------------------
""")


-------------------------------------------------------------------------------------------------------
Public Perspective: Monetized Marginal Damages from Emissions
-------------------------------------------------------------------------------------------------------
Step 1: Calculate emissions factors for different fuel sources
- Electricity
- Natural Gas
- Fuel Oil 
- Propane
      
Step 2: Adjust Natural Gas & Electricity Emissions Factors for Natural Gas Leakage
- CAMBIUM includes fugitive emissions along with other pre-combustion and combustion emissions
- Natural Gas Leakage: Deetjen et al 2021
      
Step 3: Multiply Emissions by Marginal Social Cost
- Only CO2e in the updated methods due to data quality issues for 
-------------------------------------------------------------------------------------------------------



## Electricity CO2e LRMER from CAMBIUM (includes pre-combustion (fugitive) and combustion)

### Pre-IRA Scenario: Obtain Data from Cambium 2021 (MidCase)

In [30]:
# INTERPOLATE ANNUAL DATA BETWEEN 5-YEAR TIME STEPS
print("""
-------------------------------------------------------------------------------------------------------
CLIMATE DAMAGES FROM CAMBIUM
-------------------------------------------------------------------------------------------------------
- Load CSV
- Convert MWh --> kWh and kg --> metric tons (mt)
- Inflate updated Social Cost of Carbon from $190 USD2020 to $USD2023
- Convert SCC to $USD2023/lb
- Calculate damage factors for CO2e: LRMER[lb/kWh] * SCC[$USD2023/lb] = $USD2023/kWh
- Map state, county pairs to Cambium 2023 GEA region

Possibly for other emissions projections:
- Calculate projection factors for each group of scenario and GEA region for 2025 to 2050 (normalize all annual data in group by 2025 value) 
-------------------------------------------------------------------------------------------------------
""")


-------------------------------------------------------------------------------------------------------
CLIMATE DAMAGES FROM CAMBIUM
-------------------------------------------------------------------------------------------------------
- Load CSV
- Convert MWh --> kWh and kg --> metric tons (mt)
- Inflate updated Social Cost of Carbon from $190 USD2020 to $USD2023
- Convert SCC to $USD2023/lb
- Calculate damage factors for CO2e: LRMER[lb/kWh] * SCC[$USD2023/lb] = $USD2023/kWh
- Map state, county pairs to Cambium 2023 GEA region

Possibly for other emissions projections:
- Calculate projection factors for each group of scenario and GEA region for 2025 to 2050 (normalize all annual data in group by 2025 value) 
-------------------------------------------------------------------------------------------------------



In [31]:
import os
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

print("""
-------------------------------------------------------------------------------------------------------
PRE-IRA LONG RUN AND SHORT RUN MARGINAL EMISSIONS RATES (LRMER, SRMER) FROM CAMBIUM 2021 RELEASE
-------------------------------------------------------------------------------------------------------
""")

# CAMBIUM 2021 FOR PRE-IRA SCENARIO
filename = 'cambium21_midCase_ba.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_cambium21_margEmis_electricity = pd.read_excel(io=file_path, sheet_name='cambium21_midCase_ba')

print(f"""
Retrieved data for filename: {filename}
Located at filepath: {file_path}

Loading dataframe ...
Creating lookup dictionary for LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------
""")

# Create a new DataFrame to store interpolated results
interpolated_data = []

# Group by 'scenario', 'state', and 'reeds_balancing_area'
grouped = df_cambium21_margEmis_electricity.groupby(['scenario', 'state', 'reeds_balancing_area'])

for (scenario, state, reeds_balancing_area), group in grouped:
    years = group['year'].values
    
    # Interpolate for LRMER (Long Run Marginal Emissions Rates)
    lrmer_values = group['lrmer_co2e_kg_per_MWh'].values
    lrmer_interp_func = interp1d(years, lrmer_values, kind='linear')
    
    # Interpolate for SRMER (Short Run Marginal Emissions Rates)
    srmer_values = group['srmer_co2e_kg_per_MWh'].values
    srmer_interp_func = interp1d(years, srmer_values, kind='linear')
    
    # Generate new years in 1-year increments
    new_years = np.arange(years.min(), years.max() + 1)
    
    # Interpolate the LRMER and SRMER values for these new years
    new_lrmer_values = lrmer_interp_func(new_years)
    new_srmer_values = srmer_interp_func(new_years)
    
    # Store the results in a DataFrame
    interpolated_group = pd.DataFrame({
        'scenario': scenario,
        'state': state,
        'reeds_balancing_area': reeds_balancing_area,
        'year': new_years,
        'lrmer_co2e_kg_per_MWh': new_lrmer_values,
        'srmer_co2e_kg_per_MWh': new_srmer_values
    })
    
    interpolated_data.append(interpolated_group)

# Concatenate all the interpolated data into a single DataFrame
df_cambium21_margEmis_electricity = pd.concat(interpolated_data).reset_index(drop=True)

# Convert both LRMER and SRMER values to tons per MWh and tons per kWh
df_cambium21_margEmis_electricity['lrmer_co2e_ton_per_MWh'] = df_cambium21_margEmis_electricity['lrmer_co2e_kg_per_MWh'] * (1 / 1000)
df_cambium21_margEmis_electricity['lrmer_co2e_ton_per_kWh'] = df_cambium21_margEmis_electricity['lrmer_co2e_kg_per_MWh'] * (1 / 1000) * (1 / 1000)

df_cambium21_margEmis_electricity['srmer_co2e_ton_per_MWh'] = df_cambium21_margEmis_electricity['srmer_co2e_kg_per_MWh'] * (1 / 1000)
df_cambium21_margEmis_electricity['srmer_co2e_ton_per_kWh'] = df_cambium21_margEmis_electricity['srmer_co2e_kg_per_MWh'] * (1 / 1000) * (1 / 1000)

# Display the interpolated DataFrame
df_cambium21_margEmis_electricity


-------------------------------------------------------------------------------------------------------
PRE-IRA LONG RUN AND SHORT RUN MARGINAL EMISSIONS RATES (LRMER, SRMER) FROM CAMBIUM 2021 RELEASE
-------------------------------------------------------------------------------------------------------


Retrieved data for filename: cambium21_midCase_ba.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\cambium21_midCase_ba.xlsx

Loading dataframe ...
Creating lookup dictionary for LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------



,scenario,state,reeds_balancing_area,year,lrmer_co2e_kg_per_MWh,srmer_co2e_kg_per_MWh,lrmer_co2e_ton_per_MWh,lrmer_co2e_ton_per_kWh,srmer_co2e_ton_per_MWh,srmer_co2e_ton_per_kWh
0,MidCase,AL,89,2022,703.10,877.7,0.70310,0.000703,0.8777,0.000878
1,MidCase,AL,89,2023,686.20,887.5,0.68620,0.000686,0.8875,0.000888
2,MidCase,AL,89,2024,669.30,897.3,0.66930,0.000669,0.8973,0.000897
3,MidCase,AL,89,2025,613.15,895.5,0.61315,0.000613,0.8955,0.000896
4,MidCase,AL,89,2026,557.00,893.7,0.55700,0.000557,0.8937,0.000894
...,...,...,...,...,...,...,...,...,...,...
3881,MidCase,WY,24,2046,223.60,709.7,0.22360,0.000224,0.7097,0.000710
3882,MidCase,WY,24,2047,229.30,700.0,0.22930,0.000229,0.7000,0.000700
3883,MidCase,WY,24,2048,235.00,690.3,0.23500,0.000235,0.6903,0.000690
3884,MidCase,WY,24,2049,229.20,689.0,0.22920,0.000229,0.6890,0.000689


In [32]:
# Create the nested lookup dictionary for both LRMER and SRMER in mt CO2e per MWh
emis_preIRA_cambium21_lookup = {}

# Populate the dictionary
for _, row in df_cambium21_margEmis_electricity.iterrows():
    outer_key = (row['scenario'], row['state'], row['reeds_balancing_area'])
    year = row['year']
    
    # Extract both LRMER and SRMER values in tons per kWh
    lrmer_value = row['lrmer_co2e_ton_per_kWh']
    srmer_value = row['srmer_co2e_ton_per_kWh']
    
    # Initialize the outer key if not already present
    if outer_key not in emis_preIRA_cambium21_lookup:
        emis_preIRA_cambium21_lookup[outer_key] = {}
    
    # Assign both LRMER and SRMER values in the inner dictionary for each year
    emis_preIRA_cambium21_lookup[outer_key][year] = {
        'lrmer_co2e': lrmer_value,
        'srmer_co2e': srmer_value
    }

# Display the lookup dictionary
emis_preIRA_cambium21_lookup


{('MidCase',
  'AL',
  89): {2022: {'lrmer_co2e': 0.0007031000000000001,
   'srmer_co2e': 0.0008777}, 2023: {'lrmer_co2e': 0.0006862000000000001,
   'srmer_co2e': 0.0008875}, 2024: {'lrmer_co2e': 0.0006693000000000001,
   'srmer_co2e': 0.0008973}, 2025: {'lrmer_co2e': 0.00061315,
   'srmer_co2e': 0.0008955000000000001}, 2026: {'lrmer_co2e': 0.0005570000000000001,
   'srmer_co2e': 0.0008937000000000001}, 2027: {'lrmer_co2e': 0.0005158,
   'srmer_co2e': 0.00088895}, 2028: {'lrmer_co2e': 0.00047460000000000004,
   'srmer_co2e': 0.0008842000000000001}, 2029: {'lrmer_co2e': 0.00045730000000000006,
   'srmer_co2e': 0.00087045}, 2030: {'lrmer_co2e': 0.00044,
   'srmer_co2e': 0.0008567000000000001}, 2031: {'lrmer_co2e': 0.00043595,
   'srmer_co2e': 0.0008499500000000001}, 2032: {'lrmer_co2e': 0.00043190000000000004,
   'srmer_co2e': 0.0008432000000000001}, 2033: {'lrmer_co2e': 0.00045900000000000004,
   'srmer_co2e': 0.00084375}, 2034: {'lrmer_co2e': 0.00048610000000000005,
   'srmer_co2e': 0.

### IRA-Reference Scenario: 
### Obtain 2024 Data from Cambium 2022

In [33]:
import os
import pandas as pd

print("""
-------------------------------------------------------------------------------------------------------
IRA LONG RUN AND SHORT RUN MARGINAL EMISSIONS RATES (LRMER, SRMER) FROM CAMBIUM 2022 RELEASE
-------------------------------------------------------------------------------------------------------
""")

# CAMBIUM 2022 FOR IRA SCENARIO
filename = 'cambium22_allScenarios_ba.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_cambium22_2024_margEmis_electricity = pd.read_excel(io=file_path, sheet_name='cambium22_scenarios_2024_ba')

print(f"""
Retrieved data for filename: {filename}
Located at filepath: {file_path}

Loading dataframe ...
Creating lookup dictionary for 2024 LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------
""")

# Convert kg/MWh to tons/MWh and tons/kWh for LRMER
df_cambium22_2024_margEmis_electricity['lrmer_co2e_ton_per_MWh'] = df_cambium22_2024_margEmis_electricity['lrmer_co2e_kg_per_MWh'] * (1 / 1000)
df_cambium22_2024_margEmis_electricity['lrmer_co2e_ton_per_kWh'] = df_cambium22_2024_margEmis_electricity['lrmer_co2e_kg_per_MWh'] * (1 / 1000) * (1 / 1000)

# Convert kg/MWh to tons/MWh and tons/kWh for SRMER if available
if 'srmer_co2e_kg_per_MWh' in df_cambium22_2024_margEmis_electricity.columns:
    df_cambium22_2024_margEmis_electricity['srmer_co2e_ton_per_MWh'] = df_cambium22_2024_margEmis_electricity['srmer_co2e_kg_per_MWh'] * (1 / 1000)
    df_cambium22_2024_margEmis_electricity['srmer_co2e_ton_per_kWh'] = df_cambium22_2024_margEmis_electricity['srmer_co2e_kg_per_MWh'] * (1 / 1000) * (1 / 1000)

# Display the df
df_cambium22_2024_margEmis_electricity



-------------------------------------------------------------------------------------------------------
IRA LONG RUN AND SHORT RUN MARGINAL EMISSIONS RATES (LRMER, SRMER) FROM CAMBIUM 2022 RELEASE
-------------------------------------------------------------------------------------------------------


Retrieved data for filename: cambium22_allScenarios_ba.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\cambium22_allScenarios_ba.xlsx

Loading dataframe ...
Creating lookup dictionary for 2024 LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------



,scenario,reeds_balancing_area,state,year,lrmer_co2e_kg_per_MWh,srmer_co2e_kg_per_MWh,lrmer_co2e_ton_per_MWh,lrmer_co2e_ton_per_kWh,srmer_co2e_ton_per_MWh,srmer_co2e_ton_per_kWh
0,MidCase,1,WA,2024,283.2,751.7,0.2832,0.000283,0.7517,0.000752
1,MidCase,10,CA,2024,207.6,765.1,0.2076,0.000208,0.7651,0.000765
2,MidCase,100,VA,2024,389.0,871.8,0.3890,0.000389,0.8718,0.000872
3,MidCase,101,FL,2024,288.4,775.9,0.2884,0.000288,0.7759,0.000776
4,MidCase,102,FL,2024,283.9,766.7,0.2839,0.000284,0.7667,0.000767
...,...,...,...,...,...,...,...,...,...,...
397,MidCase95by2050,95,SC,2024,331.8,867.9,0.3318,0.000332,0.8679,0.000868
398,MidCase95by2050,96,SC,2024,332.8,858.8,0.3328,0.000333,0.8588,0.000859
399,MidCase95by2050,97,NC,2024,323.1,878.6,0.3231,0.000323,0.8786,0.000879
400,MidCase95by2050,98,NC,2024,324.1,869.7,0.3241,0.000324,0.8697,0.000870


In [34]:
# Create the nested lookup dictionary for both LRMER and SRMER in mt CO2e per MWh
emis_IRA_2024_cambium22_lookup = {}

# Populate the dictionary
for _, row in df_cambium22_2024_margEmis_electricity.iterrows():
    outer_key = (row['scenario'], row['state'], row['reeds_balancing_area'])
    year = row['year']
    
    # Extract LRMER value
    lrmer_value = row['lrmer_co2e_ton_per_kWh']
    
    # Extract SRMER value if available
    srmer_value = row['srmer_co2e_ton_per_kWh'] if 'srmer_co2e_ton_per_kWh' in row else None
    
    # Initialize the outer key if not already present
    if outer_key not in emis_IRA_2024_cambium22_lookup:
        emis_IRA_2024_cambium22_lookup[outer_key] = {}
    
    # Assign the year and both LRMER and SRMER values in the inner dictionary
    emis_IRA_2024_cambium22_lookup[outer_key][year] = {
        'lrmer_co2e': lrmer_value,
        'srmer_co2e': srmer_value
    }

# Display the lookup dictionary
emis_IRA_2024_cambium22_lookup

{('MidCase',
  'WA',
  1): {2024: {'lrmer_co2e': 0.0002832, 'srmer_co2e': 0.0007517}},
 ('MidCase',
  'CA',
  10): {2024: {'lrmer_co2e': 0.0002076, 'srmer_co2e': 0.0007651}},
 ('MidCase',
  'VA',
  100): {2024: {'lrmer_co2e': 0.000389, 'srmer_co2e': 0.0008718}},
 ('MidCase',
  'FL',
  101): {2024: {'lrmer_co2e': 0.0002884, 'srmer_co2e': 0.0007759}},
 ('MidCase',
  'FL',
  102): {2024: {'lrmer_co2e': 0.0002839, 'srmer_co2e': 0.0007667}},
 ('MidCase',
  'MI',
  103): {2024: {'lrmer_co2e': 0.0004359, 'srmer_co2e': 0.0009234}},
 ('MidCase',
  'MI',
  104): {2024: {'lrmer_co2e': 0.00043680000000000005,
   'srmer_co2e': 0.0009228999999999999}},
 ('MidCase',
  'IN',
  105): {2024: {'lrmer_co2e': 0.00045400000000000003,
   'srmer_co2e': 0.0009211}},
 ('MidCase',
  'IN',
  106): {2024: {'lrmer_co2e': 0.000453, 'srmer_co2e': 0.0009214000000000001}},
 ('MidCase',
  'IN',
  107): {2024: {'lrmer_co2e': 0.0004529, 'srmer_co2e': 0.0009281000000000001}},
 ('MidCase',
  'KY',
  108): {2024: {'lrmer_co2

### Obtain 2025-2050 Data from Cambium 2023

In [35]:
print("""
-------------------------------------------------------------------------------------------------------
IRA REFERENCE SCENARIO LRMER AND SRMER (2025-2050) FROM CAMBIUM 2023 RELEASE
-------------------------------------------------------------------------------------------------------
""")

# CAMBIUM 2023 FOR IRA REFERENCE SCENARIO
filename = 'cambium23_allScenarios_ba.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_cambium23_margEmis_electricity = pd.read_excel(io=file_path, sheet_name='cambium23_allScenarios_ba')

print(f"""
Retrieved data for filename: {filename}
Located at filepath: {file_path}

Loading dataframe ...
Creating lookup dictionary for 2025-2050 LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------
""")

# Create a new DataFrame to store interpolated results
interpolated_data = []

# Group by 'scenario', 'state', and 'reeds_balancing_area'
grouped = df_cambium23_margEmis_electricity.groupby(['scenario', 'state', 'reeds_balancing_area'])

for (scenario, state, reeds_balancing_area), group in grouped:
    years = group['year'].values
    
    # Interpolate for LRMER (Long Run Marginal Emissions Rates)
    lrmer_values = group['lrmer_co2e_kg_per_MWh'].values
    lrmer_interp_func = interp1d(years, lrmer_values, kind='linear')
    
    # Interpolate for SRMER (Short Run Marginal Emissions Rates)
    srmer_values = group['srmer_co2e_kg_per_MWh'].values
    srmer_interp_func = interp1d(years, srmer_values, kind='linear')
    
    # Generate new years in 1-year increments
    new_years = np.arange(years.min(), years.max() + 1)
    
    # Interpolate the LRMER and SRMER values for these new years
    new_lrmer_values = lrmer_interp_func(new_years)
    new_srmer_values = srmer_interp_func(new_years)
    
    # Store the results in a DataFrame
    interpolated_group = pd.DataFrame({
        'scenario': scenario,
        'state': state,
        'reeds_balancing_area': reeds_balancing_area,
        'year': new_years,
        'lrmer_co2e_kg_per_MWh': new_lrmer_values,
        'srmer_co2e_kg_per_MWh': new_srmer_values
    })
    
    interpolated_data.append(interpolated_group)

# Concatenate all the interpolated data into a single DataFrame
df_cambium23_margEmis_electricity = pd.concat(interpolated_data).reset_index(drop=True)

# Convert both LRMER and SRMER values to tons per MWh and tons per kWh
df_cambium23_margEmis_electricity['lrmer_co2e_ton_per_MWh'] = df_cambium23_margEmis_electricity['lrmer_co2e_kg_per_MWh'] * (1 / 1000)
df_cambium23_margEmis_electricity['lrmer_co2e_ton_per_kWh'] = df_cambium23_margEmis_electricity['lrmer_co2e_kg_per_MWh'] * (1 / 1000) * (1 / 1000)

df_cambium23_margEmis_electricity['srmer_co2e_ton_per_MWh'] = df_cambium23_margEmis_electricity['srmer_co2e_kg_per_MWh'] * (1 / 1000)
df_cambium23_margEmis_electricity['srmer_co2e_ton_per_kWh'] = df_cambium23_margEmis_electricity['srmer_co2e_kg_per_MWh'] * (1 / 1000) * (1 / 1000)

# Display the interpolated DataFrame
df_cambium23_margEmis_electricity


-------------------------------------------------------------------------------------------------------
IRA REFERENCE SCENARIO LRMER AND SRMER (2025-2050) FROM CAMBIUM 2023 RELEASE
-------------------------------------------------------------------------------------------------------


Retrieved data for filename: cambium23_allScenarios_ba.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\cambium23_allScenarios_ba.xlsx

Loading dataframe ...
Creating lookup dictionary for 2025-2050 LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------



,scenario,state,reeds_balancing_area,year,lrmer_co2e_kg_per_MWh,srmer_co2e_kg_per_MWh,lrmer_co2e_ton_per_MWh,lrmer_co2e_ton_per_kWh,srmer_co2e_ton_per_MWh,srmer_co2e_ton_per_kWh
0,Decarb100by2035,AL,89,2025,591.60,842.80,0.59160,0.000592,0.84280,0.000843
1,Decarb100by2035,AL,89,2026,520.28,748.64,0.52028,0.000520,0.74864,0.000749
2,Decarb100by2035,AL,89,2027,448.96,654.48,0.44896,0.000449,0.65448,0.000654
3,Decarb100by2035,AL,89,2028,377.64,560.32,0.37764,0.000378,0.56032,0.000560
4,Decarb100by2035,AL,89,2029,306.32,466.16,0.30632,0.000306,0.46616,0.000466
...,...,...,...,...,...,...,...,...,...,...
27867,MidCase,WY,24,2046,122.38,122.86,0.12238,0.000122,0.12286,0.000123
27868,MidCase,WY,24,2047,152.46,120.12,0.15246,0.000152,0.12012,0.000120
27869,MidCase,WY,24,2048,182.54,117.38,0.18254,0.000183,0.11738,0.000117
27870,MidCase,WY,24,2049,212.62,114.64,0.21262,0.000213,0.11464,0.000115


In [36]:
# Create the nested lookup dictionary for LRMER and SRMER (in mt CO2e per MWh)
emis_IRA_2025_2050_cambium23_lookup = {}

# Populate the dictionary
for _, row in df_cambium23_margEmis_electricity.iterrows():
    outer_key = (row['scenario'], row['state'], row['reeds_balancing_area'])
    year = row['year']
    
    # Extract both LRMER and SRMER values in tons per kWh
    lrmer_value = row['lrmer_co2e_ton_per_kWh']
    srmer_value = row['srmer_co2e_ton_per_kWh']
    
    # Initialize the outer key if not already present
    if outer_key not in emis_IRA_2025_2050_cambium23_lookup:
        emis_IRA_2025_2050_cambium23_lookup[outer_key] = {}
    
    # Assign both LRMER and SRMER values in the inner dictionary for each year
    emis_IRA_2025_2050_cambium23_lookup[outer_key][year] = {
        'lrmer_co2e': lrmer_value,
        'srmer_co2e': srmer_value
    }

# Display the lookup dictionary
emis_IRA_2025_2050_cambium23_lookup

{('Decarb100by2035',
  'AL',
  89): {2025: {'lrmer_co2e': 0.0005916000000000001,
   'srmer_co2e': 0.0008428}, 2026: {'lrmer_co2e': 0.00052028,
   'srmer_co2e': 0.00074864}, 2027: {'lrmer_co2e': 0.00044896000000000004,
   'srmer_co2e': 0.00065448}, 2028: {'lrmer_co2e': 0.00037764,
   'srmer_co2e': 0.0005603199999999999}, 2029: {'lrmer_co2e': 0.00030632,
   'srmer_co2e': 0.00046615999999999997}, 2030: {'lrmer_co2e': 0.00023500000000000002,
   'srmer_co2e': 0.000372}, 2031: {'lrmer_co2e': 0.00018800000000000002,
   'srmer_co2e': 0.00030896}, 2032: {'lrmer_co2e': 0.000141,
   'srmer_co2e': 0.00024592000000000006}, 2033: {'lrmer_co2e': 9.400000000000001e-05,
   'srmer_co2e': 0.00018287999999999998}, 2034: {'lrmer_co2e': 4.7000000000000004e-05,
   'srmer_co2e': 0.00011984}, 2035: {'lrmer_co2e': 0.0,
   'srmer_co2e': 5.680000000000001e-05}, 2036: {'lrmer_co2e': 0.0,
   'srmer_co2e': 5.2379999999999997e-05}, 2037: {'lrmer_co2e': 0.0,
   'srmer_co2e': 4.796e-05}, 2038: {'lrmer_co2e': 0.0,
   's

## Fossil Fuels Emissions Factors (CO2e)
- RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
- Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
- All factors are in units of lb/Mbtu so energy consumption in kWh need to be converted to kWh 
- (1 lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)

In [37]:
print("""
-------------------------------------------------------------------------------------------------------
Calculate co2 and co2e Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
Fossil Fuels (Natural Gas, Fuel Oil, Propane):
- Primary Source with CO2 emissions rates
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
- Source used for equivalent CO2 emissions (CO2e)
    - NREL EUSS Webinar: https://www.osti.gov/servlets/purl/1959307
-------------------------------------------------------------------------------------------------------""")

# NATURAL GAS LEAKAGE: NATURAL GAS INFRASTRUCTURE
# leakage rate for natural gas infrastructure
# 1 Therm = 29.30 kWh --> 1.27 kg CO2e/therm * (1 therm/29.30 kWh) = 0.043 kg CO2e/kWh = 0.095 lb CO2e/kWh
naturalGas_leakage_mtCO2e_perkWh = 0.043 * (1/1000)

# Convert units from kg/MWh to ton/MWh to ton/kWh
emis_factor_co2e_naturalGas_ton_perkWh = (228.5 * (1/1000) * (1/1000)) + naturalGas_leakage_mtCO2e_perkWh
emis_factor_co2e_propane_ton_perkWh = 275.8 * (1/1000) * (1/1000)
emis_factor_co2e_fuelOil_ton_perkWh = 303.9 * (1/1000) * (1/1000)

print(f"""
Natural Gas Emissions Factor: {emis_factor_co2e_naturalGas_ton_perkWh} [mt CO2e/kWh]
Propane Emissions Factor: {emis_factor_co2e_propane_ton_perkWh} [mt CO2e/kWh]
Fuel Oil Emissions Factor: {emis_factor_co2e_fuelOil_ton_perkWh} [mt CO2e/kWh]
-------------------------------------------------------------------------------------------------------
""")


-------------------------------------------------------------------------------------------------------
Calculate co2 and co2e Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
Fossil Fuels (Natural Gas, Fuel Oil, Propane):
- Primary Source with CO2 emissions rates
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
- Source used for equivalent CO2 emissions (CO2e)
    - NREL EUSS Webinar: https://www.osti.gov/servlets/purl/1959307
-------------------------------------------------------------------------------------------------------

Natural Gas Emissions Factor: 0.00027150000000000004 [mt CO2e/kWh]
Propane Emissions Factor: 0.0002758 [mt CO2e/kWh]
Fuel Oil Emissions Factor: 0.0003039 [mt CO2e/kWh]
-----------------------------------------------------------------------------------------------------

### Step 3: Obtain CPI-U Inflation Data
- Series Id:	CUUR0000SA0
- Not Seasonally Adjusted
- Series Title:	All items in U.S. city average, all urban consumers, not seasonally adjusted
- Area:	U.S. city average
- Item:	All items
- Base Period:	1982-84=100

In [38]:
# Load the BLS Inflation Data
filename = 'bls_cpiu_2005-2023.xlsx'
relative_path = os.path.join(r"inflation_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Create a pandas dataframe
df_bls_cpiu = pd.read_excel(file_path, sheet_name='bls_cpiu')

df_bls_cpiu = pd.DataFrame({
    'year': df_bls_cpiu['Year'],
    'cpiu_annual': df_bls_cpiu['Annual']
})

# Obtain the Annual CPIU values for the years of interest
bls_cpi_annual_2008 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2008)].item()
bls_cpi_annual_2010 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2010)].item()
bls_cpi_annual_2013 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2013)].item()
bls_cpi_annual_2018 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2018)].item()
bls_cpi_annual_2019 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2019)].item()
bls_cpi_annual_2020 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2020)].item()
bls_cpi_annual_2021 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2021)].item()
bls_cpi_annual_2022 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2022)].item()
bls_cpi_annual_2023 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2023)].item()

# Precompute constant values
cpi_ratio_2023_2023 = bls_cpi_annual_2023 / bls_cpi_annual_2023
cpi_ratio_2023_2022 = bls_cpi_annual_2023 / bls_cpi_annual_2022
cpi_ratio_2023_2021 = bls_cpi_annual_2023 / bls_cpi_annual_2021  # For EPA VSL (11.3M USD-2021)
cpi_ratio_2023_2020 = bls_cpi_annual_2023 / bls_cpi_annual_2020  # For SCC
cpi_ratio_2023_2019 = bls_cpi_annual_2023 / bls_cpi_annual_2019 
cpi_ratio_2023_2018 = bls_cpi_annual_2023 / bls_cpi_annual_2018 
cpi_ratio_2023_2013 = bls_cpi_annual_2023 / bls_cpi_annual_2013
cpi_ratio_2023_2010 = bls_cpi_annual_2023 / bls_cpi_annual_2010
cpi_ratio_2023_2008 = bls_cpi_annual_2023 / bls_cpi_annual_2008  # For EPA VSL and SCC

Retrieved data for filename: bls_cpiu_2005-2023.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\inflation_data\bls_cpiu_2005-2023.xlsx


### Step 4: Use the updated Social Cost of Carbon (190 USD-2020/ton co2e) and inflate to USD-2023
- EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
- 190 USD-2020 has some inconsistency with the VSL being used. An old study and 2008 VSL is noted
- 190 USD value and inflate to USD 2023 because there is a clear source and ease of replicability.

In [39]:
# For co2e adjust SCC
epa_scc_usd2023_per_ton = 190 * cpi_ratio_2023_2020

print(f"""
Steps 3 and 4: Obtain BLS CPI-U Data and Inflate Current Social Cost of Carbon (SCC) to USD2023
      
EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
Inflate 190 $USD-2020 Social Cost of Carbon to $USD-2023

SCC Value used in analysis is: ${round(epa_scc_usd2023_per_ton, 2)} per mt CO2e
""")


Steps 3 and 4: Obtain BLS CPI-U Data and Inflate Current Social Cost of Carbon (SCC) to USD2023
      
EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
Inflate 190 $USD-2020 Social Cost of Carbon to $USD-2023

SCC Value used in analysis is: $223.69 per mt CO2e



### Step 5: Calculate End-use specific marginal damages

### Baseline Marginal Damages: WHOLE-HOME

In [40]:
print("""
-------------------------------------------------------------------------------------------------------
Step 5: Calculate End-use specific marginal damages
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Marginal Damages: WHOLE-HOME
-------------------------------------------------------------------------------------------------------
""")
# Make copies from scenario consumption to keep df smaller
print("\n", "Creating dataframe to store marginal damages calculations ...")
df_baseline_scenario_damages = df_euss_am_baseline_home.copy()

# calculate_marginal_damages(df, menu_mp, policy_scenario)
df_euss_am_baseline_home = calculate_marginal_damages(df=df_euss_am_baseline_home,
                                                      menu_mp=menu_mp,
                                                      policy_scenario='No Inflation Reduction Act',
                                                      df_summary=df_baseline_scenario_damages
                                                     )
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Step 5: Calculate End-use specific marginal damages
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Marginal Damages: WHOLE-HOME
-------------------------------------------------------------------------------------------------------


 Creating dataframe to store marginal damages calculations ...
Calculating marginal emissions and marginal damages for heating
For Marginal Emissions Factor: lrmer
For Marginal Emissions Factor: srmer
Calculating marginal emissions and marginal damages for waterHeating
For Marginal Emissions Factor: lrmer
For Marginal Emissions Factor: srmer
Calculating marginal emissions and marginal damages for clothesDrying
For Marginal Emissions Factor: lrmer
For Marginal Emissions Factor: sr

,bldg_id,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,state,city,county,...,baseline_2035_cooking_tons_co2e_srmer,baseline_2035_cooking_damages_climate_srmer,baseline_2036_cooking_tons_co2e_srmer,baseline_2036_cooking_damages_climate_srmer,baseline_2037_cooking_tons_co2e_srmer,baseline_2037_cooking_damages_climate_srmer,baseline_2038_cooking_tons_co2e_srmer,baseline_2038_cooking_damages_climate_srmer,baseline_cooking_lifetime_tons_co2e_srmer,baseline_cooking_lifetime_damages_climate_srmer
2,239,1690.0,South,East South Central,East South Central,Hot-Humid,90,AL,Not in a census Place,G0100390,...,0.29,63.95,0.29,63.89,0.29,63.81,0.28,63.74,8.40,1879.16
3,273,1690.0,South,East South Central,East South Central,Mixed-Humid,90,AL,In another census Place,G0100150,...,0.48,107.12,0.47,106.00,0.47,104.76,0.46,103.52,11.43,2556.05
4,307,1220.0,South,East South Central,East South Central,Hot-Humid,90,AL,Not in a census Place,G0100850,...,0.48,107.12,0.47,106.00,0.47,104.76,0.46,103.52,11.43,2556.05
5,409,1220.0,South,East South Central,East South Central,Hot-Humid,90,AL,Not in a census Place,G0100050,...,0.29,65.95,0.29,65.88,0.29,65.80,0.29,65.73,8.66,1937.60
7,517,1220.0,South,East South Central,East South Central,Mixed-Humid,89,AL,In another census Place,G0101270,...,0.34,74.99,0.33,74.19,0.33,73.64,0.33,73.08,8.09,1810.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548905,548109,1690.0,West,Mountain,Mountain North,Cold,23,WY,In another census Place,G5600050,...,0.34,75.22,0.33,72.73,0.33,72.72,0.33,72.70,7.36,1647.12
548907,548226,2176.0,West,Mountain,Mountain North,Cold,23,WY,In another census Place,G5600050,...,0.34,75.22,0.33,72.73,0.33,72.72,0.33,72.70,7.36,1647.12
548908,548228,1690.0,West,Mountain,Mountain North,Cold,24,WY,Not in a census Place,G5600010,...,0.33,73.59,0.32,71.21,0.32,71.23,0.32,71.25,7.25,1621.77
548910,548417,885.0,West,Mountain,Mountain North,Cold,24,WY,Casper,G5600250,...,0.39,88.27,0.38,85.42,0.38,85.44,0.38,85.46,8.70,1945.19


## Private Perspective: Annual Energy Costs

### Step 1: Obtain Level Energy Fuel Cost Data from the EIA
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm
- Propane and Fuel Oil: EIA March 2023 Short Term Energy Outlook
    - https://www.eia.gov/outlooks/steo/pdf/wf01.pdf
    - Table WF01: Average Consumer Prices and Expenditures for Heating Fuels During the Winter
    - US Average: 2018-2019 Data

In [41]:
print("""
-------------------------------------------------------------------------------------------------------
Private Perspective: Annual Energy Costs
-------------------------------------------------------------------------------------------------------
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Step 1: Obtain Level Energy Fuel Cost Data from the EIA
-------------------------------------------------------------------------------------------------------
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm
- Propane and Fuel Oil: EIA March 2023 Short Term Energy Outlook
    - https://www.eia.gov/outlooks/steo/pdf/wf01.pdf
    - Table WF01: Average Consumer Prices and Expenditures for Heating Fuels During the Winter
    - US Average: 2018-2019 Data
-------------------------------------------------------------------------------------------------------
""")

filename = 'fuel_prices_nominal.csv'
relative_path = os.path.join(r"fuel_prices", filename)
file_path = os.path.join(project_root, relative_path)
df_fuelPrices_perkWh = pd.read_csv(file_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# New units for the converted and inflated prices below
# $USD-2023, PREVIOUSLY USED $USD-2021
df_fuelPrices_perkWh['units'] = 'USD2022 per kWh'

years = ['2018', '2019', '2020', '2021', '2022']

# Take dataframe with nominal prices in their base units and convert to $/kWh equivalent
# https://www.eia.gov/energyexplained/units-and-calculators/british-thermal-units.php
for year in years:
    for index, row in df_fuelPrices_perkWh.iterrows():
        
        # Propane: (dollars per gallon) * (1 gallon propane/91,452 BTU) * (3412 BTU/1 kWh)
        if row['fuel_type'] == 'propane':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] * (1/91452) * (3412/1)
        
        # Fuel Oil: (dollars/gallon) * (1 gallon heating oil/138,500 BTU) * (3412 BTU/1 kWh)
        elif row['fuel_type'] == 'fuelOil':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] * (1/138500) * (3412/1)
        
        # Natural Gas: (dollars/cf) * (thousand cf/1000 cf) * (1 cf natural gas/1039 BTU) * (3412 BTU/1 kWh)
        elif row['fuel_type'] == 'naturalGas':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] * (1/1000) * (1/1039) * (3412/1)
        
        # Electricity: convert cents per kWh to $ per kWh
        elif row['fuel_type'] == 'electricity':
            df_fuelPrices_perkWh.at[index, f'{year}_fuelPrice_perkWh'] = row[f'{year}_nominal_unit_price'] / 100

# Convert nominal dollars to real 2022 US dollars (USD2022)
# $USD-2023, PREVIOUSLY USED $USD-2021
df_fuelPrices_perkWh['2018_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2018_fuelPrice_perkWh'] * cpi_ratio_2023_2018
df_fuelPrices_perkWh['2019_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2019_fuelPrice_perkWh'] * cpi_ratio_2023_2019
df_fuelPrices_perkWh['2020_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2020_fuelPrice_perkWh'] * cpi_ratio_2023_2020
df_fuelPrices_perkWh['2021_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2021_fuelPrice_perkWh'] * cpi_ratio_2023_2021
df_fuelPrices_perkWh['2022_fuelPrice_perkWh'] = df_fuelPrices_perkWh['2022_fuelPrice_perkWh'] * cpi_ratio_2023_2022

# Original dictionary mapping census divisions to states
map_states_census_divisions = {
    "New England": ["CT", "ME", "MA", "NH", "RI", "VT"],
    "Middle Atlantic": ["NJ", "NY", "PA"],
    "East North Central": ["IN", "IL", "MI", "OH", "WI"],
    "West North Central": ["IA", "KS", "MN", "MO", "NE", "ND", "SD"],
    "South Atlantic": ["DE", "DC", "FL", "GA", "MD", "NC", "SC", "VA", "WV"],
    "East South Central": ["AL", "KY", "MS", "TN"],
    "West South Central": ["AR", "LA", "OK", "TX"],
    "Mountain": ["AZ", "CO", "ID", "NM", "MT", "UT", "NV", "WY"],
    "Pacific": ["AK", "CA", "HI", "OR", "WA"]
}

# Reverse the mapping to create a state-to-census-division map
state_to_census_division = {}
for division, states in map_states_census_divisions.items():
    for state in states:
        state_to_census_division[state] = division

# Function to map location to census division
def map_location_to_census_division(location):
    if location in state_to_census_division:
        return state_to_census_division[location]
    return location

# Apply the function to map locations using .loc
df_fuelPrices_perkWh.loc[:, 'census_division'] = df_fuelPrices_perkWh['location_map'].apply(map_location_to_census_division)
# print(df_fuelPrices_perkWh)


-------------------------------------------------------------------------------------------------------
Private Perspective: Annual Energy Costs
-------------------------------------------------------------------------------------------------------
- Step 1: Obtain Level Energy Fuel Cost Data from the EIA
- Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Step 1: Obtain Level Energy Fuel Cost Data from the EIA
-------------------------------------------------------------------------------------------------------
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm


In [42]:
# Project Fuel Prices from 2022 to 2050
filename = 'aeo_projections_2022_2050.xlsx'
relative_path = os.path.join(r"projections", filename)
file_path = os.path.join(project_root, relative_path)
df_fuelPrices_projection_factors = pd.read_excel(io=file_path, sheet_name='fuel_price_factors_2022_2050')

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
# print(df_fuelPrices_projection_factors)

# Convert the factors dataframe into a lookup dictionary including policy_scenario
factor_dict = df_fuelPrices_projection_factors.set_index(['region', 'fuel_type', 'policy_scenario']).to_dict('index')
# print(factor_dict)

Retrieved data for filename: aeo_projections_2022_2050.xlsx
Located at filepath: c:\Users\14128\Research\cmu-tare-model\projections\aeo_projections_2022_2050.xlsx


In [43]:
# Pre-IRA policy_scenario: No Inflation Reduction Act
# Pass the desired policy_scenario as a parameter when applying the function
preIRA_projected_prices_df = df_fuelPrices_perkWh.apply(lambda row: project_future_prices(row, factor_dict, 'No Inflation Reduction Act'), axis=1)

# Concatenate the projected prices with the original DataFrame
df_fuelPrices_perkWh_preIRA = pd.concat([df_fuelPrices_perkWh, preIRA_projected_prices_df], axis=1)

# Create Fuel Price Lookup with the policy_scenario included
preIRA_fuel_price_lookup = create_fuel_price_lookup(df_fuelPrices_perkWh_preIRA, 'No Inflation Reduction Act')
# print(preIRA_fuel_price_lookup)


Processing location: National, fuel: naturalGas, policy_scenario: No Inflation Reduction Act
Initial price for 2022: 0.05043184441850111
Using projection factors for National, naturalGas, No Inflation Reduction Act: {2022: 1, 2023: 0.9755181241032465, 2024: 0.8896025800058641, 2025: 0.8412318775233192, 2026: 0.8086578203419481, 2027: 0.791217819099462, 2028: 0.7797455995454291, 2029: 0.7870977459578796, 2030: 0.7961259895028103, 2031: 0.804448058723705, 2032: 0.8090035617238965, 2033: 0.8169486175276889, 2034: 0.8214371689895306, 2035: 0.8249786745008596, 2036: 0.8264859586113565, 2037: 0.8345648475320383, 2038: 0.8507306007917632, 2039: 0.8655326974313067, 2040: 0.8655162568759125, 2041: 0.8670041621189876, 2042: 0.868808775422783, 2043: 0.869989067380462, 2044: 0.8679275616536569, 2045: 0.867906014032119, 2046: 0.8675913348058941, 2047: 0.8694052527644445, 2048: 0.8664931756653756, 2049: 0.8663586429503843, 2050: 0.867765954492123}
Year: 2022, Factor: 1, Future Price: 0.050431844418

In [44]:
# IRA-Reference policy_scenario: AEO2023 Reference Case
# Pass the desired policy_scenario as a parameter when applying the function
iraRef_projected_prices_df = df_fuelPrices_perkWh.apply(lambda row: project_future_prices(row, factor_dict, 'AEO2023 Reference Case'), axis=1)

# Concatenate the projected prices with the original DataFrame
df_fuelPrices_perkWh_iraRef = pd.concat([df_fuelPrices_perkWh, iraRef_projected_prices_df], axis=1)

# Create Fuel Price Lookup with the policy_scenario included
iraRef_fuel_price_lookup = create_fuel_price_lookup(df_fuelPrices_perkWh_iraRef, 'AEO2023 Reference Case')
# print(iraRef_fuel_price_lookup)


Processing location: National, fuel: naturalGas, policy_scenario: AEO2023 Reference Case
Initial price for 2022: 0.05043184441850111
Using projection factors for National, naturalGas, AEO2023 Reference Case: {2022: 1, 2023: 0.9839556270109511, 2024: 0.896931429801522, 2025: 0.8462875232065435, 2026: 0.8038662029823727, 2027: 0.7754636394030721, 2028: 0.7571439635072841, 2029: 0.7606949835527985, 2030: 0.7636889136297872, 2031: 0.7708472014082071, 2032: 0.7772290752131623, 2033: 0.7874921793677186, 2034: 0.7959209372991497, 2035: 0.8034350406721851, 2036: 0.8070259378095066, 2037: 0.8128780059719093, 2038: 0.8226579676773085, 2039: 0.8231216612992335, 2040: 0.8295963018124978, 2041: 0.8363521807627172, 2042: 0.8381048838873534, 2043: 0.8371994640239022, 2044: 0.8342263819703495, 2045: 0.8383924886670355, 2046: 0.860121585952518, 2047: 0.8590314022303608, 2048: 0.8567423872008247, 2049: 0.8570665110014251, 2050: 0.8594805442117667}
Year: 2022, Factor: 1, Future Price: 0.0504318444185011

### Step 2: Calculate Annual Operating (Fuel) Costs

### Baseline Fuel Cost: WHOLE-HOME

In [45]:
print("""
-------------------------------------------------------------------------------------------------------
Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
-------------------------------------------------------------------------------------------------------
""")
# calculate_annual_fuelCost(df, menu_mp, policy_scenario, drop_fuel_cost_columns)
df_euss_am_baseline_home = calculate_annual_fuelCost(df=df_euss_am_baseline_home,
                                                     menu_mp=menu_mp,
                                                     policy_scenario='No Inflation Reduction Act',
                                                     drop_fuel_cost_columns=False
                                                     )
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
-------------------------------------------------------------------------------------------------------

Calculating BASELINE (no retrofit) fuel costs from 2024 to 2039 for heating
Calculating BASELINE (no retrofit) fuel costs from 2024 to 2036 for waterHeating
Calculating BASELINE (no retrofit) fuel costs from 2024 to 2037 for clothesDrying
Calculating BASELINE (no retrofit) fuel costs from 2024 to 2039 for cooking


,bldg_id,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,state,city,county,...,baseline_2029_cooking_fuelCost,baseline_2030_cooking_fuelCost,baseline_2031_cooking_fuelCost,baseline_2032_cooking_fuelCost,baseline_2033_cooking_fuelCost,baseline_2034_cooking_fuelCost,baseline_2035_cooking_fuelCost,baseline_2036_cooking_fuelCost,baseline_2037_cooking_fuelCost,baseline_2038_cooking_fuelCost
2,239,1690.0,South,East South Central,East South Central,Hot-Humid,90,AL,Not in a census Place,G0100390,...,53.04,53.84,54.43,54.95,55.63,56.10,56.49,56.75,57.40,57.94
3,273,1690.0,South,East South Central,East South Central,Mixed-Humid,90,AL,In another census Place,G0100150,...,69.94,70.12,70.68,71.02,71.04,71.14,71.04,70.89,71.21,71.34
4,307,1220.0,South,East South Central,East South Central,Hot-Humid,90,AL,Not in a census Place,G0100850,...,69.94,70.12,70.68,71.02,71.04,71.14,71.04,70.89,71.21,71.34
5,409,1220.0,South,East South Central,East South Central,Hot-Humid,90,AL,Not in a census Place,G0100050,...,54.67,55.50,56.11,56.64,57.34,57.82,58.23,58.50,59.17,59.73
7,517,1220.0,South,East South Central,East South Central,Mixed-Humid,89,AL,In another census Place,G0101270,...,49.11,49.23,49.63,49.86,49.88,49.95,49.88,49.77,50.00,50.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548905,548109,1690.0,West,Mountain,Mountain North,Cold,23,WY,In another census Place,G5600050,...,44.12,44.61,45.36,45.95,46.28,46.52,46.62,47.06,47.46,47.71
548907,548226,2176.0,West,Mountain,Mountain North,Cold,23,WY,In another census Place,G5600050,...,44.12,44.61,45.36,45.95,46.28,46.52,46.62,47.06,47.46,47.71
548908,548228,1690.0,West,Mountain,Mountain North,Cold,24,WY,Not in a census Place,G5600010,...,44.12,44.61,45.36,45.95,46.28,46.52,46.62,47.06,47.46,47.71
548910,548417,885.0,West,Mountain,Mountain North,Cold,24,WY,Casper,G5600250,...,52.92,53.50,54.41,55.12,55.51,55.80,55.92,56.44,56.92,57.23


## Area Median Income Data Used to determine LMI Designation and IRA Rebates Eligibility/Amount

### PUMA Median Income

In [46]:
# Collect Area Median Income Data at PUMA-resolution
filename = "nhgis0003_ds261_2022_puma.csv"
relative_path = os.path.join(r"equity_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_puma_medianIncome = pd.read_csv(file_path, encoding='ISO-8859-1')
# df_puma_medianIncome = df_puma_medianIncome.drop(0)
df_puma_medianIncome = df_puma_medianIncome.reset_index(drop=True)

cols_interest = ['GISJOIN', 'STUSAB', 'PUMAA', 'NAME_E', 'AP2PE001', 'AP2PM001']
df_puma_medianIncome = df_puma_medianIncome[cols_interest]
df_puma_medianIncome = df_puma_medianIncome.rename(columns={"GISJOIN": "gis_joinID_puma", "STUSAB": "state_abbrev", "PUMAA": "puma_code", "NAME_E": "name_estimate", "AP2PE001": "median_income_USD2022", "AP2PM001": "median_income_USD2022_marginOfError"})
df_puma_medianIncome['median_income_USD2023'] = round((df_puma_medianIncome['median_income_USD2022'] * cpi_ratio_2023_2022), 2)
df_puma_medianIncome

Retrieved data for filename: nhgis0003_ds261_2022_puma.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\equity_data\nhgis0003_ds261_2022_puma.csv




,gis_joinID_puma,state_abbrev,puma_code,name_estimate,median_income_USD2022,median_income_USD2022_marginOfError,median_income_USD2023
0,G01000100,AL,100,"Lauderdale, Colbert & Franklin Counties PUMA; ...",55116.0,3490.0,57384.82
1,G01000200,AL,200,Limestone County PUMA; Alabama,84220.0,4290.0,87686.88
2,G01000300,AL,300,Morgan & Lawrence Counties--Decatur City PUMA;...,60663.0,3595.0,63160.16
3,G01000401,AL,401,Madison County (North & East)--Huntsville City...,98410.0,9589.0,102461.00
4,G01000402,AL,402,"Huntsville (North & Far West), Madison (East) ...",81364.0,4668.0,84713.31
...,...,...,...,...,...,...,...
2481,G72001900,PR,1900,"San SebastiÃ¡n, Utuado & Lares PUMA; Puerto Rico",18235.0,1281.0,18985.63
2482,G72002000,PR,2000,"Cayey, Cidra & Barranquitas PUMA; Puerto Rico",20523.0,2280.0,21367.82
2483,G72002100,PR,2100,Caguas PUMA; Puerto Rico,24929.0,3831.0,25955.19
2484,G72002200,PR,2200,Trujillo Alto & Gurabo PUMA; Puerto Rico,31984.0,6934.0,33300.61


### County Median Income

In [47]:
# Collect Area Median Income Data at PUMA-resolution
filename = "nhgis0005_ds261_2022_county.csv"
relative_path = os.path.join(r"equity_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_county_medianIncome = pd.read_csv(file_path, encoding='ISO-8859-1')
# df_county_medianIncome = df_county_medianIncome.drop(0)
df_county_medianIncome = df_county_medianIncome.reset_index(drop=True)

cols_interest = ['GISJOIN', 'STUSAB', 'COUNTYA', 'NAME_E', 'AP2PE001', 'AP2PM001']
df_county_medianIncome = df_county_medianIncome[cols_interest]
df_county_medianIncome = df_county_medianIncome.rename(columns={"GISJOIN": "gis_joinID_county", "STUSAB": "state_abbrev", "COUNTYA": "county_code", "NAME_E": "name_estimate", "AP2PE001": "median_income_USD2022", "AP2PM001": "median_income_USD2022_marginOfError"})
df_county_medianIncome['median_income_USD2023'] = round((df_county_medianIncome['median_income_USD2022'] * cpi_ratio_2023_2022), 2)
df_county_medianIncome

Retrieved data for filename: nhgis0005_ds261_2022_county.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\equity_data\nhgis0005_ds261_2022_county.csv




,gis_joinID_county,state_abbrev,county_code,name_estimate,median_income_USD2022,median_income_USD2022_marginOfError,median_income_USD2023
0,G0100030,AL,3,"Baldwin County, Alabama",69914.0,6178.0,72791.98
1,G0100150,AL,15,"Calhoun County, Alabama",52819.0,5563.0,54993.27
2,G0100430,AL,43,"Cullman County, Alabama",63229.0,5765.0,65831.79
3,G0100490,AL,49,"DeKalb County, Alabama",49663.0,3733.0,51707.35
4,G0100510,AL,51,"Elmore County, Alabama",71651.0,4293.0,74600.48
...,...,...,...,...,...,...,...
843,G7201130,PR,113,"Ponce Municipio, Puerto Rico",17700.0,1839.0,18428.61
844,G7201270,PR,127,"San Juan Municipio, Puerto Rico",25163.0,1700.0,26198.82
845,G7201350,PR,135,"Toa Alta Municipio, Puerto Rico",44584.0,7933.0,46419.28
846,G7201370,PR,137,"Toa Baja Municipio, Puerto Rico",29850.0,4241.0,31078.76


### State Median Income

In [48]:
# Collect Area Median Income Data at PUMA-resolution
filename = "nhgis0004_ds261_2022_state.csv"
relative_path = os.path.join(r"equity_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_state_medianIncome = pd.read_csv(file_path, encoding='ISO-8859-1')
# df_state_medianIncome = df_state_medianIncome.drop(0)
df_state_medianIncome = df_state_medianIncome.reset_index(drop=True)

cols_interest = ['GISJOIN', 'STUSAB','STATEA', 'NAME_E', 'AP2PE001', 'AP2PM001']
df_state_medianIncome = df_state_medianIncome[cols_interest]
df_state_medianIncome = df_state_medianIncome.rename(columns={"GISJOIN": "gis_joinID_state", "STUSAB": "state_abbrev", "STATEA": "state_code", "NAME_E": "name_estimate", "AP2PE001": "median_income_USD2022", "AP2PM001": "median_income_USD2022_marginOfError"})
df_state_medianIncome['median_income_USD2023'] = round((df_state_medianIncome['median_income_USD2022'] * cpi_ratio_2023_2022), 2)
df_state_medianIncome

Retrieved data for filename: nhgis0004_ds261_2022_state.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\equity_data\nhgis0004_ds261_2022_state.csv




,gis_joinID_state,state_abbrev,state_code,name_estimate,median_income_USD2022,median_income_USD2022_marginOfError,median_income_USD2023
0,G010,AL,1,Alabama,59674.0,727.0,62130.45
1,G020,AK,2,Alaska,88121.0,2804.0,91748.46
2,G040,AZ,4,Arizona,74568.0,932.0,77637.56
3,G050,AR,5,Arkansas,55432.0,994.0,57713.83
4,G060,CA,6,California,91551.0,471.0,95319.65
5,G080,CO,8,Colorado,89302.0,1281.0,92978.07
6,G090,CT,9,Connecticut,88429.0,1688.0,92069.14
7,G100,DE,10,Delaware,82174.0,2002.0,85556.65
8,G110,DC,11,District of Columbia,101027.0,2737.0,105185.73
9,G120,FL,12,Florida,69303.0,616.0,72155.82


### Adjustment Factors for Construction: 
#### RSMeans City Cost Index
#### Consumer Price Index for All Urban Consumers (CPI, CPI-U)

In [49]:
# Adjust for regional cost differences with RSMeans
filename = "rsMeans_cityCostIndex.csv"
relative_path = os.path.join(r"inflation_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_rsMeans_cityCostIndex = pd.read_csv(file_path)

df_rsMeans_cityCostIndex = pd.DataFrame({
    'State': df_rsMeans_cityCostIndex['State'],
    'City': df_rsMeans_cityCostIndex['City'],
    'Material': (df_rsMeans_cityCostIndex['Material']).round(2),
    'Installation': (df_rsMeans_cityCostIndex['Installation']).round(2),
    'Average': (df_rsMeans_cityCostIndex['Average']).round(2),
})
df_rsMeans_cityCostIndex

Retrieved data for filename: rsMeans_cityCostIndex.csv
Located at filepath: c:\Users\14128\Research\cmu-tare-model\inflation_data\rsMeans_cityCostIndex.csv




,State,City,Material,Installation,Average
0,United States,+30 City Average,4.20,1.20,3.00
1,Alabama,Birmingham,5.18,0.42,3.38
2,Alabama,Huntsville,4.97,0.56,3.38
3,Alabama,Mobile,5.41,0.29,3.55
4,Alabama,Montgomery,5.44,0.98,3.87
...,...,...,...,...,...
286,Wisconsin,Milwaukee,4.02,4.15,4.11
287,Wisconsin,Racine,4.96,0.00,2.76
288,Wyoming,Casper,6.04,0.00,4.05
289,Wyoming,Cheyenne,6.02,0.14,3.87


# Model Runtime

In [50]:
# Get the current datetime again
end_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Calculate the elapsed time
elapsed_time = datetime.strptime(end_time, "%Y-%m-%d_%H-%M-%S") - datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S")

# Format the elapsed time
elapsed_seconds = elapsed_time.total_seconds()
elapsed_minutes = int(elapsed_seconds // 60)
elapsed_seconds = int(elapsed_seconds % 60)

# Print the elapsed time
print(f"The code took {elapsed_minutes} minutes and {elapsed_seconds} seconds to execute.")

The code took 11 minutes and 31 seconds to execute.
